In [4]:
import pandas as pd


In [5]:
data = pd.DataFrame(columns=['title','url','id','start','end','text'])
print(data)

Empty DataFrame
Columns: [title, url, id, start, end, text]
Index: []


In [6]:
from pathlib import Path
data.to_csv(str(Path.cwd() / "data_set.csv"),sep = ',',index=False)

In [7]:
import yt_dlp
from pathlib import Path

ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': '%(title)s.%(ext)s',
        'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
        }],
        'extractaudio': True,
        'audioformat': 'mp3',
        
    }
ydl = yt_dlp.YoutubeDL(ydl_opts)
url = 'https://www.youtube.com/watch?v=J4Wdy0Wc_xQ&list=PLblh5JKOoLUIE96dI3U7oxHaCAbZgfhHk'

In [9]:

playlist_info = ydl.extract_info(url, download=False)
#For the information about videos in playlist
video_info = playlist_info.get('entries',None)

[youtube:tab] Extracting URL: https://www.youtube.com/watch?v=J4Wdy0Wc_xQ&list=PLblh5JKOoLUIE96dI3U7oxHaCAbZgfhHk
[youtube:tab] Downloading playlist PLblh5JKOoLUIE96dI3U7oxHaCAbZgfhHk - add --no-playlist to download just the video J4Wdy0Wc_xQ
[youtube:tab] PLblh5JKOoLUIE96dI3U7oxHaCAbZgfhHk: Downloading webpage
[youtube:tab] Extracting URL: https://www.youtube.com/playlist?list=PLblh5JKOoLUIE96dI3U7oxHaCAbZgfhHk
[youtube:tab] PLblh5JKOoLUIE96dI3U7oxHaCAbZgfhHk: Downloading webpage
[youtube:tab] PLblh5JKOoLUIE96dI3U7oxHaCAbZgfhHk: Redownloading playlist API JSON with unavailable videos
[download] Downloading playlist: Random Forests
[youtube:tab] Playlist Random Forests: Downloading 3 items of 3
[download] Downloading item 1 of 3
[youtube] Extracting URL: https://www.youtube.com/watch?v=J4Wdy0Wc_xQ
[youtube] J4Wdy0Wc_xQ: Downloading webpage
[youtube] J4Wdy0Wc_xQ: Downloading android player API JSON
[download] Downloading item 2 of 3
[youtube] Extracting URL: https://www.youtube.com/watc

In [10]:
import whisper
def transcribe(audio_url:str)->str:
    model = whisper.load_model("tiny")
    result = model.transcribe(audio_url,fp16=False)
    return result


In [11]:
import pandas as pd
#created empty dataframe to store
data = pd.DataFrame(columns=['title','url','id','start','end','text'])
data.to_csv(str(Path.cwd() / "data_set.csv"),sep = ',',index=False)

In [12]:
#Extracting text
for i in range(len(video_info)):
  video = video_info[i]
  url = video['webpage_url']
  ydl.download([url])
  filename = ydl.prepare_filename(video)
  filename = filename.replace('webm','mp3')
  result = transcribe(filename)
  seg = result['segments']
  df = pd.DataFrame(seg, columns=['start', 'end', 'text','id'])
  df['id'] = video['id']
  df['title'] = video['title']
  df['url'] = video['webpage_url']
  df['start'].astype('float32')
  df['end'].astype('float32')
  df = df.reindex(columns=['title','url','id','start','end','text'])
  dataset = pd.read_csv(r'C:\Users\NHI360\Desktop\ai-youtube-search\notebooks\data_set.csv')
  dataset = pd.concat([dataset,df],axis=0)
  dataset.to_csv(str(Path.cwd() / "data_set.csv"),sep = ',',index=False)
  break

[youtube] Extracting URL: https://www.youtube.com/watch?v=J4Wdy0Wc_xQ
[youtube] J4Wdy0Wc_xQ: Downloading webpage
[youtube] J4Wdy0Wc_xQ: Downloading android player API JSON
[info] J4Wdy0Wc_xQ: Downloading 1 format(s): 251
[download] StatQuest： Random Forests Part 1 - Building, Using and Evaluating.webm has already been downloaded
[download] 100% of    7.87MiB
[ExtractAudio] Destination: StatQuest： Random Forests Part 1 - Building, Using and Evaluating.mp3
Deleting original file StatQuest： Random Forests Part 1 - Building, Using and Evaluating.webm (pass -k to keep)


In [29]:
dataset = pd.read_csv(r'C:\Users\NHI360\Desktop\ai-youtube-search\notebooks\data_set.csv')
dataset = dataset.groupby(dataset.index//4).agg({'title':'first','start':'first','end':'last','text' : lambda x:''.join(x)})
dataset['duration'] = dataset['end']-dataset['start']
dataset.to_csv(str(Path.cwd() / "Final_data_set.csv"),sep = ',',index=False)

TypeError: groupby() got an unexpected keyword argument 'inplace'